Author: Omar El Malki (omar.elmalki@epfl.ch)

In [328]:
from typing import List
import re
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

from allennlp.common import JsonDict
# Semantic Role Labeling with BERT : https://github.com/Riccorl/transformer-srl
from transformer_srl import dataset_readers, models, predictors

In [12]:
def sentence_to_srl(sentence: str) -> JsonDict:
    """
    Extracts Semantic Roles from a sentence.

    :param sentence: sentence from which to extract semantic roles labels.
    :return: semantic_roles as PropBank English SRLs.
    """

    # Pre-trained model with BERT fine-tuned to predict PropBank SRLs on CoNLL 2012 dataset.
    predictor = predictors.SrlTransformersPredictor.from_path(
        "../data/pre-trained-transformer-srl/srl_bert_base_conll2012.tar.gz", "transformer_srl")

    # More documentation: https://docs.allennlp.org/models/main/models/structured_prediction/predictors/srl/
    semantic_roles: JsonDict = predictor.predict(sentence)
    return semantic_roles

In [4]:
s1 = 'Jenny lived in Florida.'
srl1=sentence_to_srl(s1)
print(f"s1=\'{s1}\', SRL={srl1}\n")

s2 = 'David noticed he had put on a lot of weight recently.'
srl2=sentence_to_srl(s2)
print(f"s2=\'{s2}\', SRL={sentence_to_srl(s2)}\n")

s1='Jenny lived in Florida.', SRL={'verbs': [{'verb': 'lived', 'description': '[ARG0: Jenny] [live.01: lived] [ARGM-LOC: in Florida] .', 'tags': ['B-ARG0', 'B-V', 'B-ARGM-LOC', 'I-ARGM-LOC', 'O'], 'frame': 'live.01', 'frame_score': 0.9999998807907104, 'lemma': 'live'}], 'words': ['Jenny', 'lived', 'in', 'Florida', '.']}

s2='David noticed he had put on a lot of weight recently.', SRL={'verbs': [{'verb': 'noticed', 'description': '[ARG0: David] [notice.01: noticed] [ARG1: he had put on a lot of weight recently] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O'], 'frame': 'notice.01', 'frame_score': 0.9999997615814209, 'lemma': 'notice'}, {'verb': 'had', 'description': 'David noticed he [have.01: had] put on a lot of weight recently .', 'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.01', 'frame_score': 1.0, 'lemma': 'have'}, {'verb': 'put', 'description': 'David noticed [ARG0

In [10]:
srl2['verbs']

[{'verb': 'noticed',
  'description': '[ARG0: David] [notice.01: noticed] [ARG1: he had put on a lot of weight recently] .',
  'tags': ['B-ARG0',
   'B-V',
   'B-ARG1',
   'I-ARG1',
   'I-ARG1',
   'I-ARG1',
   'I-ARG1',
   'I-ARG1',
   'I-ARG1',
   'I-ARG1',
   'I-ARG1',
   'O'],
  'frame': 'notice.01',
  'frame_score': 0.9999997615814209,
  'lemma': 'notice'},
 {'verb': 'had',
  'description': 'David noticed he [have.01: had] put on a lot of weight recently .',
  'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
  'frame': 'have.01',
  'frame_score': 1.0,
  'lemma': 'have'},
 {'verb': 'put',
  'description': 'David noticed [ARG0: he] had [put.08: put] on [ARG1: a lot of weight] [ARGM-TMP: recently] .',
  'tags': ['O',
   'O',
   'B-ARG0',
   'O',
   'B-V',
   'O',
   'B-ARG1',
   'I-ARG1',
   'I-ARG1',
   'I-ARG1',
   'B-ARGM-TMP',
   'O'],
  'frame': 'put.08',
  'frame_score': 0.9999700784683228,
  'lemma': 'put'}]

In [339]:
def srl_to_triple(srl: JsonDict):
    """
    Extract Knowledge triples from semantic roles

    :param srl: PropBank English SRLs
    :return: knowledge triples as a List of Lists
    """
    res = []
    verbs = srl['verbs']
    n_empty = 0
    for d in verbs:
        tags = d['tags']
        triple = d['description']
        verb = d['verb']
        if 'B-ARG0' in tags:
            arg0 = re.search('\\[ARG0: (.*?)\\]', triple).group(1)
            args = re.findall('\\[ARG[1234M]+.*?: (.*?)\\]', triple)
            for arg1 in args:  
                res.append([arg0,verb,arg1])
        elif 'B-ARG1' in tags:
            arg1 = re.search('\\[ARG1: (.*?)\\]', triple).group(1)
            args = re.findall('\\[ARG[234M]+.*?: (.*?)\\]', triple)
            for arg2 in args:  
                res.append([arg1,verb,arg2])
        else:
            n_empty += 1
            print(f"Verb with no arguments: \n{d}\n")
    if not res:
        print(f"Empty triples for SRL: \n{srl}\n")
    return res

In [340]:
srl_to_triple(srl2)

[{'verb': 'noticed', 'description': '[ARG0: David] [notice.01: noticed] [ARG1: he had put on a lot of weight recently] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O'], 'frame': 'notice.01', 'frame_score': 0.9999997615814209, 'lemma': 'notice'}, {'verb': 'had', 'description': 'David noticed he [have.01: had] put on a lot of weight recently .', 'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.01', 'frame_score': 1.0, 'lemma': 'have'}, {'verb': 'put', 'description': 'David noticed [ARG0: he] had [put.08: put] on [ARG1: a lot of weight] [ARGM-TMP: recently] .', 'tags': ['O', 'O', 'B-ARG0', 'O', 'B-V', 'O', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'O'], 'frame': 'put.08', 'frame_score': 0.9999700784683228, 'lemma': 'put'}]



[['David', 'noticed', 'he had put on a lot of weight recently'],
 ['he', 'put', 'a lot of weight'],
 ['he', 'put', 'recently']]

In [341]:
srl_to_triple(srl1)

[{'verb': 'lived', 'description': '[ARG0: Jenny] [live.01: lived] [ARGM-LOC: in Florida] .', 'tags': ['B-ARG0', 'B-V', 'B-ARGM-LOC', 'I-ARGM-LOC', 'O'], 'frame': 'live.01', 'frame_score': 0.9999998807907104, 'lemma': 'live'}]



[['Jenny', 'lived', 'in Florida']]

In [342]:
# Read ROCStories into pandas DataFrame
roc_stories_path_csv = "../data/rocstories-2017/ROCStories_winter2017.csv"
roc_stories_df = pd.read_csv(roc_stories_path_csv, sep=',', header=0)

In [346]:
srl_df = roc_stories_df.head(50)

In [347]:
# Add Knowledge triples to Dataframe for each sentence in the dataset
for n in tqdm(range(1,6)):   
    srl_df[f'srl_s{n}'] = srl_df[f'sentence{n}'].apply(lambda s: srl_to_triple(sentence_to_srl(s)))

  0%|          | 0/5 [00:00<?, ?it/s]

[{'verb': 'noticed', 'description': '[ARG0: David] [notice.01: noticed] [ARG1: he had put on a lot of weight recently] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O'], 'frame': 'notice.01', 'frame_score': 0.9999997615814209, 'lemma': 'notice'}, {'verb': 'had', 'description': 'David noticed he [have.01: had] put on a lot of weight recently .', 'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.01', 'frame_score': 1.0, 'lemma': 'have'}, {'verb': 'put', 'description': 'David noticed [ARG0: he] had [put.08: put] on [ARG1: a lot of weight] [ARGM-TMP: recently] .', 'tags': ['O', 'O', 'B-ARG0', 'O', 'B-V', 'O', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'O'], 'frame': 'put.08', 'frame_score': 0.9999700784683228, 'lemma': 'put'}]

[{'verb': 'had', 'description': '[ARG0: Tom] [have.03: had] [ARG1: a very short temper] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-AR

[{'verb': 'had', 'description': '[ARG0: Sally] [have.03: had] [ARG1: a root canal] [ARGM-TMP: this morning] , [ARGM-CAU: as she had a damaged root] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-ARGM-TMP', 'O', 'B-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'O'], 'frame': 'have.03', 'frame_score': 0.9999997615814209, 'lemma': 'have'}, {'verb': 'had', 'description': 'Sally had a root canal this morning , as [ARG0: she] [have.03: had] [ARG1: a damaged root] .', 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'O'], 'frame': 'have.03', 'frame_score': 0.9999997615814209, 'lemma': 'have'}, {'verb': 'damaged', 'description': 'Sally had a root canal this morning , as she had a [damage.01: damaged] [ARG1: root] .', 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-ARG1', 'O'], 'frame': 'damage.01', 'frame_score': 0.9999996423721313, 'lemma': 'damag

[{'verb': 'bought', 'description': '[ARG0: My roommate and I] [buy.01: bought] [ARG1: a new futon for our living room] .', 'tags': ['B-ARG0', 'I-ARG0', 'I-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O'], 'frame': 'buy.01', 'frame_score': 1.0, 'lemma': 'buy'}]

[{'verb': 'chased', 'description': '[ARG0: Bob] [chase.01: chased] [ARG1: after the rainbow] [ARGM-DIR: through the fields] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARGM-DIR', 'I-ARGM-DIR', 'I-ARGM-DIR', 'O'], 'frame': 'chase.01', 'frame_score': 0.9999954700469971, 'lemma': 'chase'}]

[{'verb': 'ran', 'description': '[ARG0: Soren] [run.02: ran] [ARGM-DIR: through the airport] , [ARGM-ADV: pulling her bags behind her] .', 'tags': ['B-ARG0', 'B-V', 'B-ARGM-DIR', 'I-ARGM-DIR', 'I-ARGM-DIR', 'O', 'B-ARGM-ADV', 'I-ARGM-ADV', 'I-ARGM-ADV', 'I-ARGM-ADV', 'I-ARGM-ADV', 'O'], 'frame': 'run.02', 'frame_score': 0.9999998807907104, 'lemma': 'run'}, {'verb': 'pulling', '

/opt/anaconda3/envs/storygen/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
 20%|██        | 1/5 [07:27<29:51, 448.00s/it]

...

[{'verb': 'felt', 'description': '[ARGM-TMP: After a few months of therapy] [ARG0: Justin] [feel.01: felt] [ARG1: better] [ARGM-LOC: around dogs] .', 'tags': ['B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-LOC', 'I-ARGM-LOC', 'O'], 'frame': 'feel.01', 'frame_score': 0.9999991655349731, 'lemma': 'feel'}]

[{'verb': 'started', 'description': '[ARGM-TMP: After a few weeks] , he [start.01: started] [ARG1: to feel much better] .', 'tags': ['B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O', 'O', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O'], 'frame': 'start.01', 'frame_score': 0.9999979734420776, 'lemma': 'start'}, {'verb': 'feel', 'description': 'After a few weeks , [ARG0: he] started to [feel.01: feel] [ARG1: much better] .', 'tags': ['O', 'O', 'O', 'O', 'O', 'B-ARG0', 'O', 'O', 'B-V', 'B-ARG1', 'I-ARG1', 'O'], 'frame': 'feel.01', 'frame_score': 0.9999986886978149, 'lemma': 'feel'}]

[{'verb': 'sat', 'd

[{'verb': 'cleared', 'description': '[ARGM-DIS: Luckily] , [ARG1: the traffic] [clear.04: cleared] up [ARGM-TMP: in time for him to get to the ceremony] .', 'tags': ['B-ARGM-DIS', 'O', 'B-ARG1', 'I-ARG1', 'B-V', 'O', 'B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O'], 'frame': 'clear.04', 'frame_score': 0.9999697208404541, 'lemma': 'clear'}, {'verb': 'get', 'description': 'Luckily , the traffic cleared up in time for [ARG1: him] to [get.05: get] [ARG2: to the ceremony] .', 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG1', 'O', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O'], 'frame': 'get.05', 'frame_score': 0.9999997615814209, 'lemma': 'get'}]

[{'verb': 'did', 'description': 'She [do.01: did] not receive another letter back in the mail .', 'tags': ['O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'do.01', 'frame_score': 0.9999996423721313, 'lemma': 'do'}, {'verb': 'receive', 'des

[{'verb': 'was', 'description': '[ARG1: Fred] [be.01: was] [ARG2: glad that his feet no longer got hot in the summer] .', 'tags': ['B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'O'], 'frame': 'be.01', 'frame_score': 1.0, 'lemma': 'be'}, {'verb': 'got', 'description': 'Fred was glad that [ARG1: his feet] [ARGM-TMP: no longer] [get.03: got] [ARG2: hot] [ARGM-TMP: in the summer] .', 'tags': ['O', 'O', 'O', 'O', 'B-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-ARGM-TMP', 'B-V', 'B-ARG2', 'B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O'], 'frame': 'get.03', 'frame_score': 0.999993085861206, 'lemma': 'get'}]

[{'verb': 'had', 'description': '[ARG0: I] [have.03: had] [ARG1: a nice time] and will visit my friend this year .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.03', 'frame_score': 0.9999996423721313, 'lemma': 'have'}, {'verb': 'will', 'description': 'I h

 80%|████████  | 4/5 [34:24<08:36, 516.05s/it]


HTTPError: 500 Server Error: Internal Server Error for url: https://huggingface.co/bert-base-cased/resolve/main/vocab.txt

In [348]:
srl_df

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,srl_s1,srl_s2,srl_s3,srl_s4
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight recently.,He examined his habits to try and figure out the reason.,He realized he'd been eating too much fast food lately.,He stopped going to burger places and started a vegetarian diet.,"After a few weeks, he started to feel much better.","[[David, noticed, he had put on a lot of weight recently], [he, put, a lot of weight], [he, put, recently]]","[[He, examined, his habits], [He, examined, to try and figure out the reason], [He, try, the reason], [He, figure, the reason]]","[[He, realized, he 'd been eating too much fast food lately], [he, eating, too much fast food], [he, eating, lately]]","[[He, stopped, going to burger places], [He, going, to burger places], [He, started, a vegetarian diet]]"
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about his actions.,"[[Tom, had, a very short temper]]","[[a guest, made, One day], [a guest, made, him very angry]]","[[He, punched, a hole in the wall of his house]]","[[Tom 's guest, became, afraid], [Tom 's guest, left, quickly]]"
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual event.,All of his clothes were either too formal or too casual.,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for the event.,"[[Marcus, needed, clothing], [Marcus, needed, for a business casual event]]","[[All of his clothes, were, either too formal or too casual]]","[[He, decided, to buy a pair of khakis], [He, buy, a pair of khakis]]","[[he, bought, The pair], [The pair he bought, fit, him], [The pair he bought, fit, perfectly]]"
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and haul it with his car.,Bill thought a truck would be better for what he needed.,Bobby pointed out two vehicles were much more expensive.,Bill was set in his ways with conventional thinking.,He ended up buying the truck he wanted despite Bobby's advice.,"[[Bobby, thought, Bill should buy a trailer and haul it with his car], [Bill, should, buy a trailer and haul], [Bill, buy, should], [Bill, buy, a trailer], [Bill, haul, should], [Bill, haul, it], [Bill, haul, with his car]]","[[Bill, thought, a truck would be better for what he needed], [a truck, be, would], [a truck, be, better for what he needed], [he, needed, what]]","[[Bobby, pointed, two vehicles were much more expensive], [two vehicles, were, much more expensive]]","[[Bill, set, in his ways], [Bill, set, with conventional thinking]]"
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in advance but to no avail.,He decided to learn to sing to overcome his handicap.,He then made all his sermons into music and sang them on Sundays.,His congregation was delighted and so was he.,"[[John, was, a pastor with a very bad memory]]","[[He, tried, to memorize his sermons many days in advance], [He, tried, but to no avail], [He, memorize, his sermons], [He, memorize, many days in advance]]","[[He, decided, to learn to sing to overcome his handicap], [He, learn, to sing], [He, learn, to overcome his handicap], [He, overcome, his handicap]]","[[He, made, then], [He, made, all his sermons into music], [He, sang, then], [He, sang, them], [He, sang, on Sundays]]"
5,4d7b022e-25d2-4300-a9b0-24ab35f4045b,Melody's trip to the aquarium.,Melody's parents surprised her with a trip to the big aquarium.,Melody took a nap during the two hour car ride to the aquarium.,"When they arrived, Melody was energetic and excited.","At the aquarium Melod

Examples of sentences with no knowledge triples.

In [355]:
# No verb detected
sentence_to_srl("That evening, the phone rang.")

{'verbs': [], 'words': ['That', 'evening', ',', 'the', 'phone', 'rang', '.']}

In [356]:
# No verb detected
sentence_to_srl("The female voice above her announced final boarding to Soren's flight.")

{'verbs': [],
 'words': ['The',
  'female',
  'voice',
  'above',
  'her',
  'announced',
  'final',
  'boarding',
  'to',
  'Soren',
  "'s",
  'flight',
  '.']}

In [357]:
# Only 1 argument
sentence_to_srl("Everybody cheered!")

{'verbs': [{'verb': 'cheered',
   'description': '[ARG0: Everybody] [cheer.02: cheered] !',
   'tags': ['B-ARG0', 'B-V', 'O'],
   'frame': 'cheer.02',
   'frame_score': 0.999954342842102,
   'lemma': 'cheer'}],
 'words': ['Everybody', 'cheered', '!']}

In [358]:
# Only 1 argument
sentence_to_srl("Eric's wife passed away.")

{'verbs': [{'verb': 'passed',
   'description': "[ARG1: Eric 's wife] [pass.16: passed] away .",
   'tags': ['B-ARG1', 'I-ARG1', 'I-ARG1', 'B-V', 'O', 'O'],
   'frame': 'pass.16',
   'frame_score': 0.9999988079071045,
   'lemma': 'pass'}],
 'words': ['Eric', "'s", 'wife', 'passed', 'away', '.']}